In [11]:
%matplotlib ipympl

import itertools
import pickle
import functools
import numpy as np
import scipy.special as sps
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook, tnrange

## Utility functions

In [12]:
"""
Fully cached memoizer decorator
from https://wiki.python.org/moin/PythonDecoratorLibrary
"""
def memoize(obj):
    cache = obj.cache = {}
    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer


"""
Poisson PDF
"""
@memoize
def poisson_prob(n, lam):
    return np.power(lam, n) / sps.factorial(n, exact=True) * np.exp(-lam)


"""
Evaluate probability of Poisson PDF for all ns >= n
"""
@memoize
def poisson_remaining_prob(n, lam):
    if n == 0:
        return 1.
    ns = np.arange(n)
    return 1. - np.sum(np.power(lam, ns) / sps.factorial(ns, exact=True) * np.exp(-lam))


## Transition functions (& state/action space dimensions)

In [13]:
MAX_NUM_CARS = 20
MAX_NUM_TRANSFERS = 5


"""
s, sp: 2x1 state - [num_cars_in_first_loc; num_cars_in_second_loc], each \in [0, MAX_NUM_CARS]
a: action \in [-MAX_NUM_TRANSFERS, MAX_NUM_TRANSFERS], representing number of cars moved from first_loc to second_loc

Returns tuple: probability, (expected) reward
"""
@memoize
def transition_prob_and_reward_ex42(s, a, sp, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS, r_transfer=-2, r_rental=10):
    nS = MAX_NUM_CARS+1
    
    r = 0
    p = 0
    
    # Check for invalid actions
    if a > 0 and s[0] < a:
        return 0., 0
    elif a < 0 and s[1] < -a:
        return 0., 0
    
    # Determine number of cars after transfer, before returns and new rentals
    r = r_transfer*abs(a)
    s_after_transfer = np.copy(s)
    s_after_transfer[0] -= a
    s_after_transfer[1] += a
    if s_after_transfer[0] > MAX_NUM_CARS:
        s_after_transfer[0] = MAX_NUM_CARS
    if s_after_transfer[1] > MAX_NUM_CARS:
        s_after_transfer[1] = MAX_NUM_CARS

    # Compute sum of probabilities by enumerating all possible returns (accumulate all remaining probs on cars=MAX_NUM_CARS) and determine corresponding number of rentals
    exp_r_rentals = 0
    for num_returns_first_loc in range(max(0, sp[0]-s_after_transfer[0]), MAX_NUM_CARS-s_after_transfer[0]+1):
        cars_after_return_first_loc = s_after_transfer[0]+num_returns_first_loc
        num_rentals_first_loc = cars_after_return_first_loc - sp[0]

        if cars_after_return_first_loc == MAX_NUM_CARS:
            prob_returns_first_loc = poisson_remaining_prob(n=num_returns_first_loc, lam=3) # more returns still result in same car state
        else:
            prob_returns_first_loc = poisson_prob(n=num_returns_first_loc, lam=3)
        if sp[0] == 0:
            prob_rentals_first_loc = poisson_remaining_prob(n=num_rentals_first_loc, lam=3) # more rentals still result in same car state
        else:
            prob_rentals_first_loc = poisson_prob(n=num_rentals_first_loc, lam=3)
        
        for num_returns_sec_loc in range(max(0, sp[1]-s_after_transfer[1]), MAX_NUM_CARS-s_after_transfer[1]+1):
            cars_after_return_sec_loc = s_after_transfer[1]+num_returns_sec_loc
            num_rentals_sec_loc = cars_after_return_sec_loc - sp[1]
            
            if cars_after_return_sec_loc == MAX_NUM_CARS:
                prob_returns_sec_loc = poisson_remaining_prob(n=num_returns_sec_loc, lam=2) # more returns still result in same car state
            else:
                prob_returns_sec_loc = poisson_prob(n=num_returns_sec_loc, lam=2)
            if sp[1] == 0:
                prob_rentals_sec_loc = poisson_remaining_prob(n=num_rentals_sec_loc, lam=4) # more rentals still result in same car state
            else:
                prob_rentals_sec_loc = poisson_prob(n=num_rentals_sec_loc, lam=4)

            prob = prob_returns_first_loc * prob_returns_sec_loc * prob_rentals_first_loc * prob_rentals_sec_loc
            p += prob
            
            exp_r_rentals += r_rental*(num_rentals_first_loc+num_rentals_sec_loc) * prob
            
            if cars_after_return_sec_loc == MAX_NUM_CARS:
                break
    
    if p > 0:
        exp_r_rentals /= p
        r += exp_r_rentals
    
    return p, r


"""
Since the transition function above does not seem to generate the same solution as the book, try one where the transition probabilities are simpler despite being wrong:
- do not consider the possibility of excessive rentals and returns
- do not normalize the poisson distributions (since we only consider a finite subset of rentals and returns, yet still use the poisson PDF defined over Z)
"""
@memoize
def transition_prob_and_reward_ex42_poissononly(s, a, sp, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS, r_transfer=-2, r_rental=10):
    nS = MAX_NUM_CARS+1
    
    r = 0
    p = 0
    
    # Check for invalid actions
    if a > 0 and s[0] < a:
        return 0., 0
    elif a < 0 and s[1] < -a:
        return 0., 0
    
    # Determine number of cars after transfer, before returns and new rentals
    r = r_transfer*abs(a)
    s_after_transfer = np.copy(s)
    s_after_transfer[0] -= a
    s_after_transfer[1] += a
    if s_after_transfer[0] > MAX_NUM_CARS:
        s_after_transfer[0] = MAX_NUM_CARS
    if s_after_transfer[1] > MAX_NUM_CARS:
        s_after_transfer[1] = MAX_NUM_CARS

    # Compute sum of probabilities by enumerating all possible returns (accumulate all remaining probs on cars=MAX_NUM_CARS) and determine corresponding number of rentals
    exp_r_rentals = 0
    for num_returns_first_loc in range(max(0, sp[0]-s_after_transfer[0]), MAX_NUM_CARS-s_after_transfer[0]+1):
        cars_after_return_first_loc = s_after_transfer[0]+num_returns_first_loc
        num_rentals_first_loc = cars_after_return_fiarst_loc - sp[0]
        prob_returns_first_loc = poisson_prob(n=num_returns_first_loc, lam=3)
        prob_rentals_first_loc = poisson_prob(n=num_rentals_first_loc, lam=3)
        
        for num_returns_sec_loc in range(max(0, sp[1]-s_after_transfer[1]), MAX_NUM_CARS-s_after_transfer[1]+1):
            cars_after_return_sec_loc = s_after_transfer[1]+num_returns_sec_loc
            num_rentals_sec_loc = cars_after_return_sec_loc - sp[1]
            prob_returns_sec_loc = poisson_prob(n=num_returns_sec_loc, lam=2)
            prob_rentals_sec_loc = poisson_prob(n=num_rentals_sec_loc, lam=4)

            prob = prob_returns_first_loc * prob_returns_sec_loc * prob_rentals_first_loc * prob_rentals_sec_loc
            p += prob
            
            exp_r_rentals += r_rental*(num_rentals_first_loc+num_rentals_sec_loc) * prob
            
            if cars_after_return_sec_loc == MAX_NUM_CARS:
                break
    
    if p > 0:
        exp_r_rentals /= p
        r += exp_r_rentals
    
    return p, r


## (Optional) Pre-cache / load-cached transition functions

In [8]:
# Pre-cache transition functions

nS = MAX_NUM_CARS+1
nA = 2*MAX_NUM_TRANSFERS+1
p_cache = np.zeros(shape=(nS, nS, nA, nS, nS))
r_cache = np.zeros(shape=(nS, nS, nA, nS, nS))
for s1 in tnrange(nS, desc='T[s1, ...]'):
    for s2, ai, s1p, s2p in itertools.product(range(nS), range(nA), range(nS), range(nS)):
        a = ai - MAX_NUM_TRANSFERS
        p_cache[s1, s2, ai, s1p, s2p], r_cache[s1, s2, ai, s1p, s2p] = transition_prob_and_reward_ex42(s=(s1, s2), a=a, sp=(s1p, s2p))

with open('pkl/Example_4.2_transition.pkl', 'wb') as f:
    pickle.dump({'fn': 'transition_prob_and_reward_ex42', 'p_cache': p_cache, 'r_cache': r_cache}, f)


HBox(children=(IntProgress(value=0, description='T[s1, ...]: ', max=21), HTML(value='')))

In [ ]:
# Load cached transition function

with open('pkl/Example_4.2_transition.pkl', 'rb') as f:
    d = pickle.load(f)
    p_cache, r_cache = d['p_cache'], d['r_cache']

def transition_prob_and_reward_ex42_cached(s, a, sp, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS):
    global p_cache, r_cache
    ai = a + MAX_NUM_TRANSFERS
    return p_cache[s[0], s[1], ai, sp[0], sp[1]], r_cache[s[0], s[1], ai, sp[0], sp[1]]


## Implementations of policy and value iteration

In [30]:
def policy_iteration(transition_prob_and_reward_fn, gamma=0.9, theta=1e-1, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS):
    nS = MAX_NUM_CARS+1
    nA = 2*MAX_NUM_TRANSFERS+1

    # Following p. 65 - Policy iteration (using iterative policy evaluation)
    # 1. Initialization
    V = np.random.rand(nS, nS)*200 + 420 # Re-scaled based on Figure 4.2
    pi = np.random.randint(low=-MAX_NUM_TRANSFERS, high=MAX_NUM_TRANSFERS+1, size=(nS, nS))
    n_iters = []

    while True:
        # 2. Policy Evaluation
        n_evals = 0
        while True:
            delta = 0
            for s1 in tnrange(nS, desc='%d-%d eval' % (len(n_iters), n_evals)):
                for s2 in range(nS):
                    v = V[s1, s2]
                    v_new = 0
                    for s1p, s2p in itertools.product(range(nS), range(nS)):
                        p, r = transition_prob_and_reward_fn(s=(s1, s2), a=pi[s1, s2], sp=(s1p, s2p))
                        v_new += p*(r + gamma*V[s1p, s2p])
                    V[s1, s2] = v_new
                    delta = max(delta, abs(v-v_new))
            n_evals += 1
            print('delta:', delta)
            if delta < theta:
                break

        # 3. Policy Improvement
        n_iters.append(n_evals)
        num_a_diff = 0
        for s1 in tnrange(nS, desc='%d impr' % len(n_iters)):
            for s2 in range(nS):
                old_action = pi[s1, s2]
                backups = np.zeros(shape=(nA,), dtype='float32')
                for ai, s1p, s2p in itertools.product(range(nA), range(nS), range(nS)):
                    a = ai - MAX_NUM_TRANSFERS
                    p, r = transition_prob_and_reward_fn(s=[s1, s2], a=a, sp=[s1p, s2p])
                    backups[ai] += p*(r + gamma*V[s1p, s2p])
                ai_max = np.argmax(backups)
                a_max = int(ai_max - MAX_NUM_TRANSFERS)
                pi[s1, s2] = a_max
                if old_action != a_max:
                    num_a_diff += 1
        print('num_a_diff',num_a_diff)
        if num_a_diff == 0:
            break
    
    return V, pi, n_iters


def value_iteration(transition_prob_and_reward_fn, gamma=0.9, theta=1e-1, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS):
    nS = MAX_NUM_CARS+1
    nA = 2*MAX_NUM_TRANSFERS+1

    # Following p. 67 - Value iteration
    # Initialization
    V = np.zeros(shape=(nS, nS), dtype='float32')
    n_iters = 0

    # Value evaluation and greedy policy evaluation
    pi = np.zeros(shape=(nS, nS), dtype='int8')
    while True:
        delta = 0
        for s1 in tnrange(nS, desc='%d impr' % n_iters):
            for s2 in range(nS):
                v = V[s1, s2]
                backups = np.zeros(shape=(nA,), dtype='float32')
                for ai, s1p, s2p in itertools.product(range(nA), range(nS), range(nS)):
                    a = ai - MAX_NUM_TRANSFERS
                    p, r = transition_prob_and_reward_fn(s=[s1, s2], a=a, sp=[s1p, s2p])
                    backups[ai] += p*(r + gamma*V[s1p, s2p])
                ai_max = np.argmax(backups)
                a_max = int(ai_max - MAX_NUM_TRANSFERS)
                pi[s1, s2] = a_max
                v_new = backups[ai_max]
                V[s1, s2] = v_new
                delta = max(delta, abs(v-v_new))
        n_iters += 1
        print('delta:', delta)
        if delta < theta:
            break
    
    return V, pi, n_iters


def q_value_iteration(transition_prob_and_reward_fn, gamma=0.9, theta=1e-1, MAX_NUM_CARS=MAX_NUM_CARS, MAX_NUM_TRANSFERS=MAX_NUM_TRANSFERS):
    nS = MAX_NUM_CARS+1
    nA = 2*MAX_NUM_TRANSFERS+1

    # Initialization
    Q = np.random.rand(nS, nS, nA)*500 # Arbitrarily-guessed scale
    n_iters = 0

    # Q-value improvement via greedy one-step backup
    while True:
        delta = 0
        for s1 in tnrange(nS, desc='%d impr' % n_iters):
            for s2, ai in itertools.product(range(nS), range(nA)):
                a = ai - MAX_NUM_TRANSFERS
                q = Q[s1, s2, ai]
                q_new = 0
                for s1p, s2p in itertools.product(range(nS), range(nS)):
                    p, r = transition_prob_and_reward_fn(s=(s1, s2), a=a, sp=(s1p, s2p))
                    q_new += p*(r + gamma*np.max(Q[s1p, s2p, :]))
                Q[s1, s2, ai] = q_new
                delta = max(delta, abs(q-q_new))
        n_iters += 1
        print('delta:', delta)
        if delta < theta:
            break

    # Extract greedy policy
    pi = np.zeros(shape=(nS, nS), dtype='int8')
    for s1, s2 in itertools.product(range(nS), range(nS)):
        ai_max = np.argmax(Q[s1, s2, :])
        a_max = int(ai_max - MAX_NUM_TRANSFERS)
        pi[s1, s2] = a_max
    
    return Q, pi, n_iters


## Policy iteration

In [25]:
# Compute policy iteration for example 4.2 using cached transition fn

V, pi, n_iters = policy_iteration(transition_prob_and_reward_ex42_cached)

with open('pkl/Exercise_4.5_cached.pkl', 'wb') as f:
    pickle.dump({'V': V, 'pi': pi, 'n_iters': n_iters}, f)


HBox(children=(IntProgress(value=0, description='0-0 eval: ', max=21), HTML(value='')))


delta: 617.004429428


HBox(children=(IntProgress(value=0, description='0-1 eval: ', max=21), HTML(value='')))


delta: 389.117163838


HBox(children=(IntProgress(value=0, description='0-2 eval: ', max=21), HTML(value='')))

...


num_a_diff 9


HBox(children=(IntProgress(value=0, description='4-0 eval: ', max=21), HTML(value='')))


delta: 0.0767491469892


HBox(children=(IntProgress(value=0, description='5 impr: ', max=21), HTML(value='')))


num_a_diff 0


In [19]:
# Compute policy iteration for example 4.2

V, pi, n_iters = policy_iteration(transition_prob_and_reward_ex42)

with open('pkl/Exercise_4.5.pkl', 'wb') as f:
    pickle.dump({'V': V, 'pi': pi, 'n_iters': n_iters}, f)


HBox(children=(IntProgress(value=0, description='0-0 eval: ', max=21), HTML(value='')))


delta: 611.08552984


HBox(children=(IntProgress(value=0, description='0-1 eval: ', max=21), HTML(value='')))


delta: 322.037483578


HBox(children=(IntProgress(value=0, description='0-2 eval: ', max=21), HTML(value='')))

...


num_a_diff 4


HBox(children=(IntProgress(value=0, description='6-0 eval: ', max=21), HTML(value='')))


delta: 0.0258878875298


HBox(children=(IntProgress(value=0, description='7 impr: ', max=21), HTML(value='')))


num_a_diff 0


In [ ]:
# Compute policy iteration for example 4.2 using simpler but technically incorrect transition function

V, pi, n_iters = policy_iteration(transition_prob_and_reward_ex42_poissononly)

with open('pkl/Exercise_4.5_poissononly.pkl', 'wb') as f:
    pickle.dump({'V': V, 'pi': pi, 'n_iters': n_iters}, f)


## Value iteration

In [31]:
# Compute value iteration for example 4.2 using cached transition fn

V, pi, n_iters = value_iteration(transition_prob_and_reward_ex42_cached)

with open('pkl/Exercise_4.5_V_cached.pkl', 'wb') as f:
    pickle.dump({'V': V, 'pi': pi, 'n_iters': n_iters}, f)


HBox(children=(IntProgress(value=0, description='0 impr: ', max=21), HTML(value='')))


delta: 299.22


HBox(children=(IntProgress(value=0, description='1 impr: ', max=21), HTML(value='')))


delta: 109.47


HBox(children=(IntProgress(value=0, description='2 impr: ', max=21), HTML(value='')))

...


delta: 0.12973


HBox(children=(IntProgress(value=0, description='38 impr: ', max=21), HTML(value='')))


delta: 0.108704


HBox(children=(IntProgress(value=0, description='39 impr: ', max=21), HTML(value='')))


delta: 0.0913391


In [17]:
# Compute value iteration for example 4.2

V, pi, n_iters = value_iteration(transition_prob_and_reward_ex42)

with open('pkl/Exercise_4.5_V.pkl', 'wb') as f:
    pickle.dump({'V': V, 'pi': pi, 'n_iters': n_iters}, f)


HBox(children=(IntProgress(value=0, description='impr 0 s1: ', max=21), HTML(value='')))


delta: 299.22


HBox(children=(IntProgress(value=0, description='impr 1 s1: ', max=21), HTML(value='')))


delta: 109.47


HBox(children=(IntProgress(value=0, description='impr 2 s1: ', max=21), HTML(value='')))

...


delta: 0.12973


HBox(children=(IntProgress(value=0, description='impr 38 s1: ', max=21), HTML(value='')))


delta: 0.108704


HBox(children=(IntProgress(value=0, description='impr 39 s1: ', max=21), HTML(value='')))


delta: 0.0913391


## Q-value iteration

In [33]:
# Compute Q-value iteration for example 4.2 using cached transition fn

Q, pi, n_iters = q_value_iteration(transition_prob_and_reward_ex42_cached)

with open('pkl/Exercise_4.5_Q_cached.pkl', 'wb') as f:
    pickle.dump({'Q': Q, 'pi': pi, 'n_iters': n_iters}, f)


HBox(children=(IntProgress(value=0, description='0 impr: ', max=21), HTML(value='')))


delta: 540.28938369


HBox(children=(IntProgress(value=0, description='1 impr: ', max=21), HTML(value='')))


delta: 66.9504961917


HBox(children=(IntProgress(value=0, description='2 impr: ', max=21), HTML(value='')))

...


delta: 0.137648475686


HBox(children=(IntProgress(value=0, description='25 impr: ', max=21), HTML(value='')))


delta: 0.115381432758


HBox(children=(IntProgress(value=0, description='26 impr: ', max=21), HTML(value='')))


delta: 0.0967666822984


In [ ]:
# Compute Q-value iteration for example 4.2

Q, pi, n_iters = q_value_iteration(transition_prob_and_reward_ex42)

with open('pkl/Exercise_4.5_Q.pkl', 'wb') as f:
    pickle.dump({'Q': Q, 'pi': pi, 'n_iters': n_iters}, f)


## Visualizations

In [34]:
# Visualize final value function and policy

#results_filename = 'pkl/Exercise_4.5.pkl'
#results_filename = 'pkl/Exercise_4.5_cached.pkl'
#results_filename = 'pkl/Exercise_4.5_onlypoisson.pkl'
#results_filename = 'pkl/Exercise_4.5_V.pkl'
results_filename = 'pkl/Exercise_4.5_V_cached.pkl'
with open(results_filename, 'rb') as f:
    d = pickle.load(f)
    V, pi, n_iters = d['V'], d['pi'], d['n_iters']
    
plt.matshow(V, origin='lower')
plt.xlabel('Number of cars in first location')
plt.ylabel('Number of cars in second location')
plt.title('Value function')

plt.matshow(pi, origin='lower')
plt.xlabel('Number of cars in first location')
plt.ylabel('Number of cars in second location')
plt.title('Policy')

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5,1.05,'Policy')

In [35]:
# Visualize final Q-value functions and policy

#results_filename = 'pkl/Exercise_4.5_Q.pkl'
results_filename = 'pkl/Exercise_4.5_Q_cached.pkl'
with open(results_filename, 'rb') as f:
    d = pickle.load(f)
    Q, pi, n_iters = d['Q'], d['pi'], d['n_iters']
    
for ai, a in enumerate(range(-MAX_NUM_TRANSFERS, MAX_NUM_TRANSFERS+1)):
    plt.matshow(Q[:, :, ai], origin='lower')
    plt.xlabel('Number of cars in first location')
    plt.ylabel('Number of cars in second location')
    plt.title('Q-Value function for a=%d' % a)

plt.matshow(pi, origin='lower')
plt.xlabel('Number of cars in first location')
plt.ylabel('Number of cars in second location')
plt.title('Policy function')

/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

...

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0.5,1.05,'Policy function')

## TODOs
- exercise 4.5
- proper tqdm